# Projeto 1 - Ciência dos Dados

Nome:Christian Delfs

Nome:Eduardo Zetone

Nome: Guilherme Pinheiro


Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from math import *
import warnings
warnings.filterwarnings('ignore')

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [2]:
filename = 'Heineken.xlsx'

In [3]:
train = pd.read_excel(filename)
train.tail()

,Treinamento,Relevância
746,que vontade sentar em uma mesa de bar e tomar ...,4
747,abraçadinha no fardinho de heineken q será dev...,3
748,12 heineken e minha cabeça já tá doendo.\nisso...,2
749,heineken é mt ruim 🤡🤡,0
750,que vontade de tomar uma #heineken 😍😍,4


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.tail(5)

,Teste,Relevância
246,heineken pior cerveja,0
247,comigo agora é assim: quer me beijar? me pague...,4
248,@gugall1 um fardo de heineken e dou pra vocês ...,3
249,"fazendo as contas,bebemos 10 baldes de heineke...",2
250,heineken é mó boa slc,4


In [5]:
train['Treinamento'] = train['Treinamento'].astype('str')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...
Heineken é uma cerveja, muito popular no Brasil
palavras relevantes:
trincando, não da ressaca, bom, saudades, geladinha
palavras não relevantes:
ruim, caro, merda 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
def cleanup(text):
    punctuation = '[!-.:?;"\n"()''"",_%$\|/,<>]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    filtro = filter(lambda x:x[0]!='@', text_subbed.split())
    new = " ".join(filter(lambda x:x[0]!='@', text_subbed.split()))
    return new
train['Treinamento'] = train['Treinamento'].apply(cleanup).str.lower()

In [7]:
train['Relevância'] = train['Relevância'].astype('category')
train['Relevância'].cat.categories = ['Muito irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito relevante']

In [8]:
#codigo de filtro
muito_relevante = train['Relevância'] == 'Muito relevante'
relevante = train['Relevância'] == 'Relevante'
neutro = train['Relevância'] == 'Neutro'
irrelevante = train['Relevância'] == 'Irrelevante'
muito_irrelevante = train['Relevância'] == 'Muito irrelevante'
#df dos filtros
df_muito_relevante = train.loc[muito_relevante, :]
df_relevante = train.loc[relevante, :]
df_neutro = train.loc[neutro, :]
df_irrelevante = train.loc[irrelevante, :]
df_muito_irrelevante = train.loc[muito_irrelevante, :]
tudo=[df_muito_relevante['Treinamento'],df_relevante['Treinamento'],df_neutro['Treinamento'],df_irrelevante['Treinamento'],df_muito_irrelevante['Treinamento']]

In [9]:
#código que tranforma df em pd.series
tweets_muito_rel = []
tweets_rel = []
tweets_neu = []
tweets_irr = []
tweets_muito_irr = []
tudo2=[tweets_muito_rel,tweets_rel,tweets_neu,tweets_irr,tweets_muito_irr]
Todo= []
Mundo = []
Odeia = []
O= []
Chris = []
tudo3=[Todo,Mundo,Odeia,O,Chris]
i=0
while i<len(tudo):
    for a in tudo[i]:
        tudo2[i].append(a.split(' '))
    i+=1
ded=0
while ded<len(tudo2):
    for Eduzin in tudo2[ded]:
         tudo3[ded]+= Eduzin
    ded+=1

serie_muito_relevante=pd.Series(Todo)
serie_relevante=pd.Series(Mundo)
serie_neutro=pd.Series(Odeia)
serie_irrelevante=pd.Series(O)
serie_muito_irrelevante=pd.Series(Chris)

In [10]:
#utilizamos a variante Laplace Smoothing (alpha = 1)
alpha = 1

v= 1e3

In [11]:
freq_ab_mr = serie_muito_relevante.value_counts() 
freq_re_mr = (freq_ab_mr + alpha)/(freq_ab_mr.sum() + (alpha*v))

freq_ab_r = serie_relevante.value_counts() 
freq_re_r = (freq_ab_r + alpha)/(freq_ab_r.sum() + (alpha*v))

freq_ab_n = serie_neutro.value_counts() 
freq_re_n = (freq_ab_n + alpha)/(freq_ab_n.sum() + (alpha*v))

freq_ab_i = serie_irrelevante.value_counts() 
freq_re_i = (freq_ab_i + alpha)/(freq_ab_i.sum() + (alpha*v)) 

freq_ab_mi = serie_muito_irrelevante.value_counts() 
freq_re_mi = (freq_ab_mi + alpha)/(freq_ab_mi.sum() + (alpha*v))
tudo3=[freq_ab_mr,freq_ab_r,freq_ab_n,freq_ab_i,freq_ab_mi]
tudo4=[freq_re_mr,freq_re_r,freq_re_n,freq_re_i,freq_re_mi]

In [12]:
p_mr = len(tweets_muito_rel) / len(train["Treinamento"])
p_r = len(tweets_rel) / len(train["Treinamento"])
p_n = len(tweets_neu) / len(train["Treinamento"])
p_i = len(tweets_irr) / len(train["Treinamento"])
p_mi = len(tweets_muito_irr) / len(train["Treinamento"])
tudo5=[p_mr,p_r,p_n,p_i,p_mi]

In [13]:
def funça(frase):
    frase = frase.split(' ')
    prob_frase_dado_mr = 1
    prob_frase_dado_r = 1
    prob_frase_dado_n = 1
    prob_frase_dado_i = 1
    prob_frase_dado_mi = 1
    tudo6=[prob_frase_dado_mr,prob_frase_dado_r,prob_frase_dado_n,prob_frase_dado_i,prob_frase_dado_mi]
    
    prob_mr_dado_frase = 1
    prob_r_dado_frase = 1
    prob_n_dado_frase = 1
    prob_i_dado_frase = 1
    prob_mi_dado_frase = 1
    tudo8=[prob_mr_dado_frase,prob_r_dado_frase,prob_n_dado_frase,prob_i_dado_frase,prob_mi_dado_frase]
    
    tudo9=['Muito relevante','Relevante','Neutro','Irrelevante','Muito irrelevante']
    
    for palavra in frase:
        for a in range(5):
            if palavra in tudo4[a]:
                tudo6[a] *= tudo4[a][palavra]
            elif palavra not in tudo4[a]:
                tudo6[a] *= (alpha/(tudo3[a].sum() + (alpha*v)))
    for a in range(5):
        tudo8[a]=(tudo6[a])*(tudo5[a])
    for b in range(5):
        if max(log(tudo8[0], 10), log(tudo8[1], 10), log(tudo8[2], 10), log(tudo8[3], 10),  log(tudo8[4], 10)) == log(tudo8[b], 10):
            return tudo9[b]

In [14]:
train['Classificacao'] = train['Treinamento'].apply(funça)
train['Classificacao']

0            Irrelevante
1                 Neutro
2      Muito irrelevante
3        Muito relevante
4        Muito relevante
             ...        
746      Muito relevante
747            Relevante
748               Neutro
749    Muito irrelevante
750      Muito relevante
Name: Classificacao, Length: 751, dtype: object

In [15]:
tabela = pd.crosstab(train['Relevância'], train['Classificacao'], normalize='all')
tabela

Classificacao,Irrelevante,Muito irrelevante,Muito relevante,Neutro,Relevante
Relevância,,,,,
Muito irrelevante,0.001332,0.138482,0.010652,0.003995,0.005326
Irrelevante,0.037284,0.001332,0.003995,0.006658,0.003995
Neutro,0.000000,0.001332,0.014647,0.121172,0.006658
Relevante,0.001332,0.001332,0.027963,0.005326,0.201065
Muito relevante,0.002663,0.001332,0.388815,0.001332,0.011984


In [16]:
#quanto o classificador acertou?
f'{(tabela.iloc[0,1] + tabela.iloc[1,0] + tabela.iloc[2,3] + tabela.iloc[3,4] + tabela.iloc[4,2])*100:.5g}%'

'88.682%'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [17]:
teste = pd.read_excel(filename, sheet_name = 'Teste')
teste['Teste'] = teste['Teste'].astype('str')
test['Teste'] = teste['Teste'].apply(cleanup).str.lower()

In [18]:
teste['Relevância'] = teste['Relevância'].astype('category')
teste['Relevância'].cat.categories = ['Muito irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito relevante']
teste['Classificador'] = teste['Teste'].apply(funça)

In [19]:
tabela2 = pd.crosstab(teste['Relevância'], teste['Classificador'], normalize='all')
tabela2

Classificador,Irrelevante,Muito irrelevante,Muito relevante,Neutro,Relevante
Relevância,,,,,
Muito irrelevante,0.067729,0.075697,0.035857,0.055777,0.035857
Irrelevante,0.019920,0.027888,0.019920,0.027888,0.023904
Neutro,0.039841,0.035857,0.083665,0.011952,0.031873
Relevante,0.035857,0.011952,0.095618,0.027888,0.055777
Muito relevante,0.019920,0.011952,0.111554,0.011952,0.023904


In [20]:
f'O acerto do nosso classificador foi de: {(tabela2.iloc[0,1]+tabela2.iloc[1,0]+tabela2.iloc[2,3]+tabela2.iloc[3,4]+tabela2.iloc[4,2])*100:.5g}%'

'O acerto do nosso classificador foi de: 27.49%'

___
### Concluindo

### Por que não posso usar o próprio classificador para gerar mais amostras de treinamento?

Não podemos usar o nosso classificador para gerar mais amostras de treinamento porque foi feito a partir das classificações de treinamento. Assim a eficácia do classificador se mostra ser muito alta, não condizendo com a realidade, já que os tweets que serão classificados geraram os 'padrões' que criaram o classifcador.

### Diferentes cenários para Naïve Bayes fora do contexto do projeto


O classificador Naïve-Bayes também pode ser usado para saber a rejeição de uma pessoa, por exemplo quando um famoso vai para o BBB e é julgado por pessoas do lado de fora da casa, através das classificações das mensagens que o mencionam em qualquer rede social. De modo geral, o classificador Naïve-Bayes pode ser muito útil para realizar uma análise automática de sentimento relacionada a qualquer produto, marca ou pessoa. É uma maneira de obter feedbacks de forma informal, sincera e aleatória.

### Melhorias possíveis para o classificador Naïve-Bayes


Considerando a ordem em que as palavras aparecem na frase e a probabilidade de certas sequências de palavras, podemos obter uma classificação mais realista. Com isso, podemos analisar a relevância, positividade e categorias negativas da mensagem pela probabilidade de que uma determinada sequência se enquadre em qualquer um desses cenários.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**